In [116]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from pandas.core.common import flatten
from itertools import zip_longest

import numpy as np
import re

import gensim
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pymorphy2
!pip install googletrans
from googletrans import Translator


In [117]:
result = pd.read_csv('result.csv', sep=';', names=['param','guid','group','result'])
train = pd.read_csv('train.csv', sep=';', names=['param','guid','group'])
classes = pd.read_csv('classes.csv', sep=';', names=['guid','param','group'])

In [118]:
classes

,guid,param,group
0,4289cb19-9517-45de-9c02-5a74ebf5c86d,ADSK_Единица измерения,TEXT
1,a8cdbf7b-d60a-485e-a520-447d2055f351,ADSK_Завод-изготовитель,TEXT
2,c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,ADSK_Зона,TEXT
3,2fd9e8cb-84f3-4297-b8b8-75f444e124ed,ADSK_Код изделия,TEXT
4,2204049c-d557-4dfc-8d70-13f19715e46d,ADSK_Марка,TEXT
...,...,...,...
241,14e630a8-bc4f-4556-9094-647e8f323f08,ADSK_Размер_Глубина,LENGTH
242,d6888dc7-7a03-40c3-9ac2-ab300c4e2c0a,ADSK_Размер_ДиаметрИзделия,LENGTH
243,398ea174-e0ef-4619-8db9-0b2c598d5206,ADSK_Размер_Толщина основы,LENGTH
244,a7397d18-200b-4659-b34c-3d8ae1c54317,ADSK_Размер_УголПоворота,ANGLE


In [119]:
# pd.merge(train, classes, ho)

In [120]:
train

,param,guid,group
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer
...,...,...,...
1001,Мощность,6f6a6a94-13ad-4946-ba66-fac6566f0033,HVACPower
1002,Толщина 1,736dabe1-ea07-4213-ba84-afe395de2820,PipeSize
1003,"Тип, марка, обозначение документа, опросного л...",90214fe6-6311-44cc-be60-55b6a63c6418,Text
1004,"Масса единицы, кг",9e2f55f6-15fe-4892-af03-4a0dc62bd468,Text


In [121]:
result.head()

,param,guid,group,result
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,ADSK_Количество фаз
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,ADSK_Единица измерения
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,ADSK_Единица измерения
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,ADSK_Классификация нагрузок
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,ADSK_Количество фаз


In [122]:
pd.merge(train, result, how='right', left_on='guid', right_on='guid').head()

,param_x,guid,group_x,param_y,group_y,result
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,Количество фаз,NumberOfPoles,ADSK_Количество фаз
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,Ед. измерения,Text,ADSK_Единица измерения
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,Единица измерения,Text,ADSK_Единица измерения
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,Классификация нагрузок,LoadClassification,ADSK_Классификация нагрузок
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,n_фаз,Integer,ADSK_Количество фаз


In [123]:
classes.head()

,guid,param,group
0,4289cb19-9517-45de-9c02-5a74ebf5c86d,ADSK_Единица измерения,TEXT
1,a8cdbf7b-d60a-485e-a520-447d2055f351,ADSK_Завод-изготовитель,TEXT
2,c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,ADSK_Зона,TEXT
3,2fd9e8cb-84f3-4297-b8b8-75f444e124ed,ADSK_Код изделия,TEXT
4,2204049c-d557-4dfc-8d70-13f19715e46d,ADSK_Марка,TEXT


## НУЖНО РАСПЛИТИТЬ ПО БУКВАМ CLASSES и TRAIN

In [124]:
def split_phase(phase):
    for f, s in zip_longest(phase, phase[1:]):
        if f.isalpha() and s is None:
            word += f
            yield word
            word = ''
        elif f.isalpha() and s.isalpha() and f.isupper() and s.islower():
            # начало слова: первая заглавная, вторая маленькая
            word = ''
            word += f
        elif f.isalpha() and s.isalpha() and f.islower() and s.islower():
            # слово продолжается, если две маленькие
            word += f
        elif f.isalpha() and s.isalpha() and f.islower() and s.isupper():
            # слова заканчивается если переход с маленькой на заглавную
            word += f
            yield word
            word = ''
        elif f.isalpha() and s.isalpha() and f.isupper() and s.isupper():
            word += f
        elif f.isalpha() and s.isalpha() == False:
            try:
                word += f
            except UnboundLocalError:
                word = f
            yield word
            word = ''

In [125]:
re.findall('[A-Z][^A-Z]*', 'TheLongAndWindingRoad')
re.findall('[A-Z][^A-Z]*', 'TTT')
re.findall('[А-ЯЁ][^А-ЯЁ]*', 'УГО УголПоворота')

# re.findall('[А-ЯЁ][^А-ЯЁ]*', 'УголПоворота')
# re.findall('[^А-ЯЁ]*', 'УГО')
# withoutregex('УГооа')

['У', 'Г', 'О ', 'Угол', 'Поворота']

In [126]:
classes['source'] = classes['param']
classes['param'] = classes['param'].str.split('_').apply(lambda x: x[1:])


In [127]:
classes.head()
classes[classes.source == 'ADSK_Примечание']

,guid,param,group,source
14,a85b7661-26b0-412f-979c-66af80b4b2c3,[Примечание],TEXT,ADSK_Примечание


### Функция обработки

In [128]:
def splitting(Ser):
    Ser = Ser.apply(lambda x: [re.split(';|,|\*|\n|\s|_|-', y) for y in x])
    Ser = Ser.apply(lambda x: [re.findall('[А-ЯЁ][^А-ЯЁ]*', y) for y in list(flatten(x))])
    Ser = Ser.apply(lambda x: list(flatten(x)))
    Ser = Ser.apply(lambda x: [y.lower() for y in x if (len(y) != 1) and (y.isdigit() is not True)])
#     Ser = Ser.str.join(' ')
#     Ser = Ser.apply(lambda x: x[0] if type(x) == list else x)
    return Ser

In [129]:
def splitting_s(Ser):
    Ser = Ser.apply(lambda x:  [re.findall('[А-ЯЁ]{2,}', y) for y in x])
#     Ser = Ser.apply(lambda x: [re.split(';|,|\*|\n|\s|_|-', y) for y in x])
#     Ser = Ser.apply(lambda x: list(flatten(x)))
    Ser = Ser.apply(lambda x: list(flatten(x)))
    Ser = Ser.apply(lambda x: [y.lower() for y in x if (len(y) != 1) and (y.isdigit() is not True)])
#     Ser = Ser.str.join(' ')
    return Ser

In [130]:
def splitting_f(Ser):
    Ser = Ser.apply(lambda x: [re.split(';|,|\*|\n|\s|_|-', y) for y in x])
    Ser = Ser.apply(lambda x: list(flatten(x)))
    Ser = Ser.apply(lambda x: [y.lower() for y in x if (len(y) != 1) and (y.isdigit() is not True)])
#     Ser = Ser.str.join(' ')
    return Ser

In [131]:
def splitting_en(Ser):
    Ser = Ser.apply(lambda x: [re.split(';|,|\*|\n|\s|_|-', y) for y in x])
    Ser = Ser.apply(lambda x: [re.findall('[A-Z][^A-Z]*', y) for y in list(flatten(x))])
    Ser = Ser.apply(lambda x: list(flatten(x)))
    Ser = Ser.apply(lambda x: [y.lower() for y in x if (len(y) != 1) and (y.isdigit() is not True)])
#     Ser = Ser.str.join(' ')
#     Ser = Ser.apply(lambda x: x[0] if type(x) == list else x)
    return Ser

In [132]:
def splitting_s_en(Ser):
    Ser = Ser.apply(lambda x:  [re.findall('[A-Z]{2,}', y) for y in x])
#     Ser = Ser.apply(lambda x: [re.split(';|,|\*|\n|\s|_|-', y) for y in x])
    Ser = Ser.apply(lambda x: list(flatten(x)))
    Ser = Ser.apply(lambda x: [y.lower() for y in x if (len(y) != 1) and (y.isdigit() is not True)])
#     Ser = Ser.str.join(' ')
    return Ser

In [133]:
def common(Ser):
    return (splitting(Ser) + splitting_s(Ser) + splitting_en(Ser) + splitting_s_en(Ser) + splitting_f(Ser)).apply(lambda x: ' '.join(list(set(x))))

In [134]:
test2 = ['короткое наименование','короткое наименование']
test = pd.Series(['n_фаз', 'короткое наименование', 'URL УГО разработчика', 'ТестТестировщик', 'Product Page URL'])
test = test.apply(lambda x: [x])
# test.apply(lambda x: [re.findall('[a-zA-z]+', y) for y in x])
out1 = splitting(test)
out2 = splitting_s(test)
out3 = splitting_en(test)
out4 = splitting_s_en(test)
out5 = splitting_f(test)
# joing(out1, out2, out3, out4)
(out1 + out2 + out3 + out4 + out5).apply(lambda x: ' '.join(list(set(x))))
common(test)
# re.findall('[А-ЯЁ][^А-ЯЁ]*', 'n_фаз')
# test2

0                                 фаз
1               наименование короткое
2                разработчика уго url
3    тесттестировщик тест тестировщик
4                    product page url
dtype: object

### Обрабатываем classes

In [135]:
# classes['test'] = classes['param'].apply(lambda x: ' '.join(x))
# classes['test']
# pd.Series(list(flatten(classes['test'].apply(lambda x: list(split_phase(x))).to_list()))).value_counts()

In [136]:
# classes['test'].apply(lambda x: list(split_phase(x))).to_list()

In [137]:
classes.head()

,guid,param,group,source
0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения
1,a8cdbf7b-d60a-485e-a520-447d2055f351,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель
2,c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,[Зона],TEXT,ADSK_Зона
3,2fd9e8cb-84f3-4297-b8b8-75f444e124ed,[Код изделия],TEXT,ADSK_Код изделия
4,2204049c-d557-4dfc-8d70-13f19715e46d,[Марка],TEXT,ADSK_Марка


In [138]:
## новая функция


In [139]:
## старая функция
classes['vector_ADSK'] = common(classes['param'])
print(len(classes))
# classes['vector'].apply(type).value_counts()
classes


246


,guid,param,group,source,vector_ADSK
0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерения
1,a8cdbf7b-d60a-485e-a520-447d2055f351,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель,завод изготовитель
2,c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,[Зона],TEXT,ADSK_Зона,зона
3,2fd9e8cb-84f3-4297-b8b8-75f444e124ed,[Код изделия],TEXT,ADSK_Код изделия,код изделия
4,2204049c-d557-4dfc-8d70-13f19715e46d,[Марка],TEXT,ADSK_Марка,марка
...,...,...,...,...,...
241,14e630a8-bc4f-4556-9094-647e8f323f08,"[Размер, Глубина]",LENGTH,ADSK_Размер_Глубина,глубина размер
242,d6888dc7-7a03-40c3-9ac2-ab300c4e2c0a,"[Размер, ДиаметрИзделия]",LENGTH,ADSK_Размер_ДиаметрИзделия,размер диаметр диаметризделия изделия
243,398ea174-e0ef-4619-8db9-0b2c598d5206,"[Размер, Толщина основы]",LENGTH,ADSK_Размер_Толщина основы,основы размер толщина
244,a7397d18-200b-4659-b34c-3d8ae1c54317,"[Размер, УголПоворота]",ANGLE,ADSK_Размер_УголПоворота,поворота угол размер уголповорота


In [140]:
classes[classes.source == 'ADSK_Примечание']

,guid,param,group,source,vector_ADSK
14,a85b7661-26b0-412f-979c-66af80b4b2c3,[Примечание],TEXT,ADSK_Примечание,примечание


In [141]:
pd.Series(list(flatten(classes['vector_ADSK'].to_list()))).value_counts()

                                  10
должность боковой строка штамп     6
фамилия строка штамп               6
должность строка штамп             6
фамилия боковой строка штамп       6
                                  ..
отверстие от этажа отметка         1
помещения категория                1
титул фамилия строка               1
элемента узла типоразмер           1
параметр незаданный                1
Length: 210, dtype: int64

In [142]:
# classes['vector'] = classes['vector'].str.join(' ')
classes['param'].iloc[0]

['Единица измерения']

In [143]:
classes =classes.reset_index()
classes.head()

,index,guid,param,group,source,vector_ADSK
0,0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерения
1,1,a8cdbf7b-d60a-485e-a520-447d2055f351,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель,завод изготовитель
2,2,c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,[Зона],TEXT,ADSK_Зона,зона
3,3,2fd9e8cb-84f3-4297-b8b8-75f444e124ed,[Код изделия],TEXT,ADSK_Код изделия,код изделия
4,4,2204049c-d557-4dfc-8d70-13f19715e46d,[Марка],TEXT,ADSK_Марка,марка


### Обрабатываем train

In [144]:
# train['param'] = train['param'].str.split()
train.head()

,param,guid,group
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer


In [145]:
train['param'] = train['param'].apply(lambda x: [x])
train['vector'] = common(train['param'])
train.head()

,param,guid,group,vector
0,[Количество фаз],a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,количество фаз
1,[Ед. измерения],763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,измерения ед.
2,[Единица измерения],9e75a863-eed0-4d52-a37d-49b818375f45,Text,единица измерения
3,[Классификация нагрузок],799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,классификация нагрузок
4,[n_фаз],0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,фаз


In [146]:
translator = Translator()
# result = translator.translate('количество фаз', dest='ru')
# train['vector'].apply(lambda x: translator.translate(x, dest='ru').text if translator.translate(x, dest='ru').src != 'ru')



# train['vector'] = train['vector'].apply(lambda x: translator.translate(x, dest='ru').text if translator.translate('x', dest='ru').src != 'ru' else x)
train['vector'] = train['vector'].apply(lambda x: translator.translate(x, dest='ru').text)

# translator.translate('количество фаз', dest='ru').src != 'ru'

In [147]:
# test = train['vector'].apply(lambda x: translator.translate(x, dest='ru').text)

In [148]:
pd.DataFrame(train)
# train['vector'] = common(train['param'])
# train.head()

,param,guid,group,vector
0,[Количество фаз],a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,количество фаз
1,[Ед. измерения],763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,измерения ед.
2,[Единица измерения],9e75a863-eed0-4d52-a37d-49b818375f45,Text,единица измерения
3,[Классификация нагрузок],799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,классификация нагрузок
4,[n_фаз],0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,фаз
...,...,...,...,...
1001,[Мощность],6f6a6a94-13ad-4946-ba66-fac6566f0033,HVACPower,мощность
1002,[Толщина 1],736dabe1-ea07-4213-ba84-afe395de2820,PipeSize,толщина
1003,"[Тип, марка, обозначение документа, опросного ...",90214fe6-6311-44cc-be60-55b6a63c6418,Text,документа листа обозначение тип опросного марка
1004,"[Масса единицы, кг]",9e2f55f6-15fe-4892-af03-4a0dc62bd468,Text,кг масса единицы


In [149]:
train[train.guid == '8519c1cf-831f-4680-9177-29283900e20a']

,param,guid,group,vector
864,[Width],8519c1cf-831f-4680-9177-29283900e20a,Length,width


In [150]:
train[train.guid == '0d72a99b-a015-4c23-9ceb-1c8bbff1995a']


,param,guid,group,vector
902,[Product data url],0d72a99b-a015-4c23-9ceb-1c8bbff1995a,URL,product data url


In [151]:
classes[classes.guid == 'a7397d18-200b-4659-b34c-3d8ae1c54317']

,index,guid,param,group,source,vector_ADSK
244,244,a7397d18-200b-4659-b34c-3d8ae1c54317,"[Размер, УголПоворота]",ANGLE,ADSK_Размер_УголПоворота,поворота угол размер уголповорота


In [152]:
test = classes.copy()
test['param'].apply(lambda x: list(map(str.lower, x)))

0           [единица измерения]
1          [завод-изготовитель]
2                        [зона]
3                 [код изделия]
4                       [марка]
                 ...           
241           [размер, глубина]
242    [размер, диаметризделия]
243    [размер, толщина основы]
244      [размер, уголповорота]
245            [размер, радиус]
Name: param, Length: 246, dtype: object

In [153]:

test[test.guid == 'a7397d18-200b-4659-b34c-3d8ae1c54317']

,index,guid,param,group,source,vector_ADSK
244,244,a7397d18-200b-4659-b34c-3d8ae1c54317,"[Размер, УголПоворота]",ANGLE,ADSK_Размер_УголПоворота,поворота угол размер уголповорота


In [154]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def check_fake(list_words):
    for word in list_words:
        yield morph.parse(word)[0].normal_form


morph.parse("марки для высота установки")[0].normal_form
morph.parse("уголповорота")[0].methods_stack
morph.parse("марки для высота установки")[0].methods_stack[0][0]
list(check_fake(test['vector_ADSK'].str.split().iloc[244]))    #['поворота', 'уголповорота', 'угол', 'размер']

test['vector_ADSK'].str.split().apply(lambda x: list(check_fake(x)))
## применить к train и classes


0                            [единица, измерение]
1                           [завод, изготовитель]
2                                          [зона]
3                                  [код, изделие]
4                                         [марка]
                          ...                    
241                             [глубина, размер]
242    [размер, диаметр, диаметризделие, изделие]
243                     [основа, размер, толщина]
244          [поворот, угол, размер, уголповорот]
245                              [размер, радиус]
Name: vector_ADSK, Length: 246, dtype: object

In [155]:
train.head()
train['vector'] = train['vector'].str.split().apply(lambda x: list(check_fake(x)))
classes.head()
classes['vector_ADSK'] = classes['vector_ADSK'].str.split().apply(lambda x: list(check_fake(x)))
classes.head()

,index,guid,param,group,source,vector_ADSK
0,0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения,"[единица, измерение]"
1,1,a8cdbf7b-d60a-485e-a520-447d2055f351,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель,"[завод, изготовитель]"
2,2,c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,[Зона],TEXT,ADSK_Зона,[зона]
3,3,2fd9e8cb-84f3-4297-b8b8-75f444e124ed,[Код изделия],TEXT,ADSK_Код изделия,"[код, изделие]"
4,4,2204049c-d557-4dfc-8d70-13f19715e46d,[Марка],TEXT,ADSK_Марка,[марка]


In [156]:
classes['vector_ADSK'] = classes['vector_ADSK'].str.join(' ')
train['vector'] = train['vector'].str.join(' ')

### Создаем модель и обучаем на Classes (ADSK параметрах)

In [157]:
text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier())
                     ])

In [158]:
text_clf.fit(classes['vector_ADSK'], classes['index'])

C:\Users\r.enikeev\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

In [159]:
train.head()

,param,guid,group,vector
0,[Количество фаз],a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,количество фаза
1,[Ед. измерения],763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,измерение ед.
2,[Единица измерения],9e75a863-eed0-4d52-a37d-49b818375f45,Text,единица измерение
3,[Классификация нагрузок],799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,классификация нагрузка
4,[n_фаз],0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,фаза


In [160]:
train[train.guid == '0257a4b0-6a8d-42c3-bae6-334ab79a9818']
train['vector'][train['vector'].str.split().apply(len) > 3]

11                                  тот гост (ту) у)
32                 по сортировка раздел спецификация
49               единица измерение измерение единица
75                    рейтинг проектный критерий шум
77                 критерий оценка шум запланировать
                            ...                     
997                                 по уго ося длина
998                                по уго ося ширина
1000                      марка высота для установка
1003    документ лист обозначение тип опросный марка
1005               код оборудование материал изделие
Name: vector, Length: 219, dtype: object

## ПРЕДИКТ

In [161]:
res = text_clf.predict(train['vector'])
train['predict'] = res
train.head()

,param,guid,group,vector,predict
0,[Количество фаз],a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,количество фаза,54
1,[Ед. измерения],763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,измерение ед.,0
2,[Единица измерения],9e75a863-eed0-4d52-a37d-49b818375f45,Text,единица измерение,0
3,[Классификация нагрузок],799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,классификация нагрузка,53
4,[n_фаз],0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,фаза,128


In [162]:
train[train.guid == 'c3a92f4c-a74d-4c9e-8a27-369811a97586']
len(train)

1006

In [163]:
pd.merge(train, classes, how='inner', left_on='guid', right_on='guid')

,param_x,guid,group_x,vector,predict,index,param_y,group_y,source,vector_ADSK
0,[О_Масса],32989501-0d17-4916-8777-da950841c6d7,Number,масса,5,5,[Масса],NUMBER,ADSK_Масса,масса
1,[О_Примечание],a85b7661-26b0-412f-979c-66af80b4b2c3,Text,примечание,14,14,[Примечание],TEXT,ADSK_Примечание,примечание
2,[О_Комплект],8dd021be-382d-4776-afd4-75996e351de3,Text,комплект,83,83,[Комплект],TEXT,ADSK_Комплект,комплект
3,[О_Обозначение],9c98831b-9450-412d-b072-7d69b39f4029,Text,обозначение,11,11,[Обозначение],TEXT,ADSK_Обозначение,обозначение
4,[О_Наименование],e6e0f5cd-3e26-485b-9342-23882b20eb43,Text,наименование,9,9,[Наименование],TEXT,ADSK_Наименование,наименование
5,[О_КодИзделия],2fd9e8cb-84f3-4297-b8b8-75f444e124ed,Text,код изделие кодизделие,3,3,[Код изделия],TEXT,ADSK_Код изделия,код изделие
6,[И_ЗаводИзготовитель],a8cdbf7b-d60a-485e-a520-447d2055f351,Text,завод заводизготовитель изготовитель,1,1,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель,завод изготовитель
7,[О_Этаж],9eabf56c-a6cd-4b5c-a9d0-e9223e19ea3f,Text,этаж,15,15,[Этаж],TEXT,ADSK_Этаж,этаж
8,[О_ЕдиницыИзмерения],4289cb19-9517-45de-9c02-5a74ebf5c86d,Text,единица измерение измерение единица,0,0,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерение
9,[О_Зона],c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,Text,зона,2,2,[Зона],TEXT,ADSK_Зона,зона


In [164]:
classes.head()

,index,guid,param,group,source,vector_ADSK
0,0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерение
1,1,a8cdbf7b-d60a-485e-a520-447d2055f351,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель,завод изготовитель
2,2,c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,[Зона],TEXT,ADSK_Зона,зона
3,3,2fd9e8cb-84f3-4297-b8b8-75f444e124ed,[Код изделия],TEXT,ADSK_Код изделия,код изделие
4,4,2204049c-d557-4dfc-8d70-13f19715e46d,[Марка],TEXT,ADSK_Марка,марка


In [165]:
pred = pd.merge(train, classes, how='left', left_on='predict', right_on='index')
pred.head()


,param_x,guid_x,group_x,vector,predict,index,guid_y,param_y,group_y,source,vector_ADSK
0,[Количество фаз],a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,количество фаза,54,54,d182b385-9e45-4e8b-b8da-725396848493,[Количество фаз],NOOFPOLES,ADSK_Количество фаз,количество фаза
1,[Ед. измерения],763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,измерение ед.,0,0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерение
2,[Единица измерения],9e75a863-eed0-4d52-a37d-49b818375f45,Text,единица измерение,0,0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерение
3,[Классификация нагрузок],799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,классификация нагрузка,53,53,a3413727-1213-401f-8ff2-880522a42b91,[Классификация нагрузок],LOADCLASSIFICATION,ADSK_Классификация нагрузок,классификация нагрузка
4,[n_фаз],0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,фаза,128,128,16dfbbd1-6716-417e-bcb8-2f46e9e82b6a,[D],REINFORCEMENT_LENGTH,ADSK_D,


In [166]:
train[train['guid'] == '5bceeb6e-a0d2-43f2-94a6-7081d89be3b3']

,param,guid,group,vector,predict
30,[PF_Угол],5bceeb6e-a0d2-43f2-94a6-7081d89be3b3,Angle,pf угол,128


In [179]:
classes[classes['source'] == 'ADSK_Размер_УголПоворота']

,index,guid,param,group,source,vector_ADSK
244,244,a7397d18-200b-4659-b34c-3d8ae1c54317,"[Размер, УголПоворота]",ANGLE,ADSK_Размер_УголПоворота,поворот угол размер уголповорот


In [168]:
train[train['guid'] == 'a2d06669-a727-4e50-bcba-d7bb905d3372']

,param,guid,group,vector,predict
0,[Количество фаз],a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,количество фаза,54


In [169]:
pred.head()

,param_x,guid_x,group_x,vector,predict,index,guid_y,param_y,group_y,source,vector_ADSK
0,[Количество фаз],a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,количество фаза,54,54,d182b385-9e45-4e8b-b8da-725396848493,[Количество фаз],NOOFPOLES,ADSK_Количество фаз,количество фаза
1,[Ед. измерения],763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,измерение ед.,0,0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерение
2,[Единица измерения],9e75a863-eed0-4d52-a37d-49b818375f45,Text,единица измерение,0,0,4289cb19-9517-45de-9c02-5a74ebf5c86d,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерение
3,[Классификация нагрузок],799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,классификация нагрузка,53,53,a3413727-1213-401f-8ff2-880522a42b91,[Классификация нагрузок],LOADCLASSIFICATION,ADSK_Классификация нагрузок,классификация нагрузка
4,[n_фаз],0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,фаза,128,128,16dfbbd1-6716-417e-bcb8-2f46e9e82b6a,[D],REINFORCEMENT_LENGTH,ADSK_D,


In [170]:
pred = pred[['param_x', 'guid_x', 'source', 'vector', 'vector_ADSK']]
pred.columns = ['source', 'guid', 'predict_p', 'vector', 'vector_ADSK']
pred[pred.guid == 'c3a92f4c-a74d-4c9e-8a27-369811a97586']
# pred

,source,guid,predict_p,vector,vector_ADSK
665,[Примечания],c3a92f4c-a74d-4c9e-8a27-369811a97586,ADSK_Примечание,примечание,примечание


In [171]:
result

,param,guid,group,result
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,ADSK_Количество фаз
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,ADSK_Единица измерения
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,ADSK_Единица измерения
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,ADSK_Классификация нагрузок
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,ADSK_Количество фаз
...,...,...,...,...
120,Ширина,5bee8e1a-2928-4464-ad47-2f05d24d8505,Length,ADSK_Размер_Ширина
121,Номер помещения,d0efb795-ce89-4a3c-a7f7-191acf6e5504,Text,ADSK_Номер помещения квартиры
122,Мощность,6f6a6a94-13ad-4946-ba66-fac6566f0033,HVACPower,ADSK_Тепловая мощность
123,"Масса единицы, кг",9e2f55f6-15fe-4892-af03-4a0dc62bd468,Text,ADSK_Масса


In [172]:
result.head()
len(result)
len(pred)

1006

In [173]:
analyz = pd.merge(result, pred, how='left', left_on='guid', right_on='guid')
analyz['true'] = (analyz['result'] == analyz['predict_p'])
analyz.head()


,param,guid,group,result,source,predict_p,vector,vector_ADSK,true
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,ADSK_Количество фаз,[Количество фаз],ADSK_Количество фаз,количество фаза,количество фаза,True
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,ADSK_Единица измерения,[Ед. измерения],ADSK_Единица измерения,измерение ед.,единица измерение,True
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,ADSK_Единица измерения,[Единица измерения],ADSK_Единица измерения,единица измерение,единица измерение,True
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,ADSK_Классификация нагрузок,[Классификация нагрузок],ADSK_Классификация нагрузок,классификация нагрузка,классификация нагрузка,True
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,ADSK_Количество фаз,[n_фаз],ADSK_D,фаза,,False


In [174]:
analyz[['param', 'guid', 'result', 'predict_p', 'vector','vector_ADSK', 'true']].to_excel('mytry.xlsx')

In [175]:
sum(analyz['result'] == analyz['predict_p'])/len(analyz)

0.696

In [ ]:
result
pred
len(pd.merge(result, pred, how='left', left_on='guid', right_on='guid'))
# len(ready), len(result)
pd.merge(result, pred, how='inner', left_on='guid', right_on='guid')

In [221]:
classes

,guid,param,group
0,4289cb19-9517-45de-9c02-5a74ebf5c86d,ADSK_Единица измерения,TEXT
1,a8cdbf7b-d60a-485e-a520-447d2055f351,ADSK_Завод-изготовитель,TEXT
2,c78f0a7d-b68b-4d21-a247-1c8c6ced8bc5,ADSK_Зона,TEXT
3,2fd9e8cb-84f3-4297-b8b8-75f444e124ed,ADSK_Код изделия,TEXT
4,2204049c-d557-4dfc-8d70-13f19715e46d,ADSK_Марка,TEXT
...,...,...,...
241,14e630a8-bc4f-4556-9094-647e8f323f08,ADSK_Размер_Глубина,LENGTH
242,d6888dc7-7a03-40c3-9ac2-ab300c4e2c0a,ADSK_Размер_ДиаметрИзделия,LENGTH
243,398ea174-e0ef-4619-8db9-0b2c598d5206,ADSK_Размер_Толщина основы,LENGTH
244,a7397d18-200b-4659-b34c-3d8ae1c54317,ADSK_Размер_УголПоворота,ANGLE


In [282]:
analyz[analyz['guid'] == '106f2a8b-3006-4864-aaf0-d07ea744b6f1']

,param,guid,group,result,source,predict_p,vector,vector_ADSK,true
101,Width,106f2a8b-3006-4864-aaf0-d07ea744b6f1,Length,ADSK_Размер_Ширина,[Width],ADSK_G,width,,False
